This notebook downloads the 5 input features for the whole Colombia.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=puazGZKDW_MlhD6mK5uN4R7JaaWUkAC8jzAbh2dE_iQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2QHGE0FaFu4CAXMDIMD6OFdWiUClDVUFxS8qYyLM_J3nfRbFo_y9wik

Successfully saved authorization token.


In [4]:
catalog = {
    'vegetation': ('MODIS/006/MOD13A2', 'EVI'),
    'aridity': ('MODIS/006/MOD16A2', 'ET'), # 500m
    'temperature': ('MODIS/006/MOD11A1', 'LST_Day_1km'),
    'nighttime_lights': ('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG', 'avg_rad'),
    'population': ('WorldPop/GP/100m/pop', 'population'), #100m
}

features = list(catalog.keys())

In [ ]:
def gee_median(
        BBOX, 
        FILENAME = 'gee_sample',
        year = 2018, 
        min_dt = None, 
        max_dt = None,
        PRODUCT = None,
        BAND = 'B0',
        verbose = 1,
    ):
    'adapted from sen2median https://github.com/thinkingmachines/geoai-immap/blob/master/utils/gee.py#L156'
    
    # set date window
    if (min_dt is None) | (max_dt is None):
        date1 = f'{year}-01-01'
        date2 = f'{year}-12-31'
    else:
        date1 = min_dt
        date2 = max_dt
    
    # select region
    region = ee.Geometry.Rectangle(BBOX) # restrict view to bounding box

    #obtain the image
    global composite
    composite = (ee.ImageCollection(PRODUCT)
     .filterDate(date1, date2)
     .filterBounds(region)
     .select(BAND)
     .median())
    
    # Export task
    task = ee.batch.Export.image.toCloudStorage(
        image=composite,
        description= FILENAME,
        bucket= 'immap-gee',
        maxPixels= 150000000,
        scale= 1000,
        region= region,
        crs= 'EPSG:4326'
    )


    task.start()
    print('Task started')

In [ ]:
# BBOX = [-72.292152, 11.734492, -72.244001, 11.686520] # small uribia urban area
# BBOX = [-73.17020892181104, 11.560920839000062, -72.52724612099996, 10.948171764015513] # riohacha
BBOX = [-81.7389221191406,13.3974609375,-66.8377380371094,-4.22784757614136] # whole colombia
area = 'colombia'
year = 2018

print(f'Starting for {area}')
for feature in features:
    product, band = catalog[feature]
    fname = f'gee_wash_{year}_{area}_{feature}'
    print(f'Downloading {feature} with product ID {product} to {fname}')
    gee_median(
            BBOX, 
            FILENAME = fname,
            year = year,
            PRODUCT = product,
            BAND = band,
            verbose = 1,
        )

Starting for colombia
Task started
Task started
Task started
Task started
Task started


In [7]:
for feature in features:
    !gsutil cp gs://immap-gee/gee_wash_{year}_{area}_{feature}.tif gs://immap-wash-training/features/{year}_{area}_{feature}.tif

Copying gs://immap-gee/gee_wash_2018_colombia_vegetation.tif [Content-Type=image/tiff]...
/ [1 files][  5.6 MiB/  5.6 MiB]                                                
Operation completed over 1 objects/5.6 MiB.                                      
Copying gs://immap-gee/gee_wash_2018_colombia_aridity.tif [Content-Type=image/tiff]...
/ [1 files][  3.8 MiB/  3.8 MiB]                                                
Operation completed over 1 objects/3.8 MiB.                                      
Copying gs://immap-gee/gee_wash_2018_colombia_temperature.tif [Content-Type=image/tiff]...
/ [1 files][  5.0 MiB/  5.0 MiB]                                                
Operation completed over 1 objects/5.0 MiB.                                      
Copying gs://immap-gee/gee_wash_2018_colombia_nighttime_lights.tif [Content-Type=image/tiff]...
/ [1 files][  5.6 MiB/  5.6 MiB]                                                
Operation completed over 1 objects/5.6 MiB.                       